In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [12]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        # ROI 설정 (x, y, width, height)
        roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
        
        # ROI 추출
        roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

        # 그레이스케일 변환
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        # 원본 프레임에 엣지 결과를 덮어쓰기 (ROI 영역만)
        frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w] = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        
        if run: # Frame을 읽은 경우
            # 결과 출력
            cv2.imshow('Frame with ROI Edges', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [13]:
# Video가 저장된 경로 입력
PATH = r".\plank_sample2.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [14]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
print(f"- Maximum Intensity: {video.max()}")
print(f"- Minimum Intensity: {video.min()}")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (79, 480, 852, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [15]:
def find_contours(morphed):
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_keypoints(image, keypoints):
    for point in keypoints:
        cv2.circle(image, point, 5, (0, 0, 255), -1)
    return image

def detect_hip_position(contours):
    keypoints = []
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # 작은 윤곽선 무시
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                keypoints.append((cX, cY))
    
    if len(keypoints) > 0:
        keypoints = sorted(keypoints, key=lambda k: k[1])  # y좌표 기준 정렬
        hip = keypoints[len(keypoints) // 2]  # 중간 값이 엉덩이
        return hip
    return None


In [16]:

def draw_bounding_boxes(frame, contours, min_width, max_width, min_height, max_height):
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if min_width <= w <= max_width and min_height <= h <= max_height:
            bounding_boxes.append((x, y, w, h))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return frame, bounding_boxes


def binarize_image(image, threshold=127):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return binary

def count_white_pixels(edge_image):
    return np.sum(edge_image == 255)

def calculate_frame_difference(frame1, frame2):
    diff = cv2.absdiff(frame1, frame2)
    binary_diff = binarize_image(diff)
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(binary_diff, cv2.MORPH_ERODE, kernel, iterations=1)
    return morphed, count_white_pixels(binary_diff)


In [20]:
import cv2
import numpy as np
import time

# 영상 읽기
cap = cv2.VideoCapture('plank_sample2.mp4')

# 타이머 초기화
timer_running = False
start_time = 0
elapsed_time = 0

# 흰색 픽셀 임계값 설정
white_pixel_threshold = 30

bg_subtractor = cv2.createBackgroundSubtractorMOG2()
frame_count = 0
_, prev_frame = cap.read()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    h, w, c = frame.shape
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 3)
    fg_mask = bg_subtractor.apply(blurred)
    _, binary = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # morphology 연산
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(binary, cv2.MORPH_ERODE, kernel, iterations=1)
    # morphed = cv2.morphologyEx(morphed, cv2.MORPH_OPEN, kernel, iterations=2)

    if frame_count % 50 == 0:
        edges_diff, white_pixel_count = calculate_frame_difference(prev_frame, frame)
            
        # 타이머 상태 제어
        if white_pixel_count > 2000:
            print("자세 교정!")
        else:
            prev_frame = frame.copy()
        
        # 차분 이미지를 별도로 표시
        cv2.imshow('Frame Difference', edges_diff)
        print(white_pixel_count)
        

    # Canny 엣지 검출
    edges = cv2.Canny(blurred, 50, 150)

    # 흰색 픽셀 개수 세기
    white_pixel_count = np.sum(morphed == 255)
    
    
    # 타이머 상태 제어
    if white_pixel_count > white_pixel_threshold:
        if timer_running:
            # 타이머 멈춤
            elapsed_time += time.time() - start_time
            timer_running = False
    else:
        if not timer_running:
            # 타이머 작동
            start_time = time.time()
            timer_running = True
    
    # 현재 타이머 상태 표시
    if timer_running:
        current_time = time.time()
        display_time = elapsed_time + (current_time - start_time)
    else:
        display_time = elapsed_time
    
    # 타이머 표시
    cv2.putText(frame, f'Timer: {display_time:.2f} seconds', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0) if timer_running else (0, 0, 255), 2)


    # cv2.imshow('Plank Pose Monitoring', binary[h//3:, w*2//5:w*3//5])
    cv2.imshow('Plank Pose Monitoring2', frame)
    # cv2.imshow('Plank Pose Monitoring', morphed)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


자세 교정!
2210
1450
0
2
0
